In [ ]:
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'scroll': True,
        'width': "100%",
        'height': "100%",
})

# SOLID Workshop
- SRE: Single Responsibility Principle
- OCP: Open Close Principle
- LSP: Liskov Substitution Principle
- ISP:
- DIP:

## SRE: Single Responsibility Principle

"Every software compponent should have one and only one responsibility"

Key points:
- Aim for High Cohesion 
- Aim for Loose Coupling

### Example 1: Cohesion 

"Cohesion is the degree to which the various parts of a software components are related"

In [ ]:
class Square:
    def __init__(self):
        self.side = 5
    
    def calculate_area(self):
        return self.side * self.side
    
    def calculate_perimeter(self):
        return self.side * 4
    
    def draw(self):
        print("rendering the square image")
    
    def rotate(self, degree):
        print(f"rotating the image of the square clockwise to {degree} degree")
        self.draw()

In [ ]:
square = Square()
print(f"Square Area: {square.calculate_area()}")
print(f"Square Perimeter: {square.calculate_perimeter()}")

square.draw()
square.rotate(25)

### Let's refactor the code

In [ ]:
class Square:
    def __init__(self):
        self.side:int = 5
    
    def calculate_area(self) -> int:
        return self.side * self.side
    
    def calculate_perimeter(self) -> int:
        return self.side * 4

In [ ]:
class SquareUI:
    def __init__(self, square:Square):
        self.square: Square = square
        
    def draw(self):
        print("rendering the square image")
    
    def rotate(self, degree:int):
        print(f"rotating the image of the square clockwise to {degree} degree")
        self.draw()

In [ ]:
square = Square()
print(f"Square Area: {square.calculate_area()}")
print(f"Square Perimeter: {square.calculate_perimeter()}")

square_ui = SquareUI(square)
square_ui.draw()
square_ui.rotate(25)

### Example 2: Coupling 

"Coupling is defined as the level of inter-dependency between various software components"

In [ ]:
import mysql.connector

class Student:
    def __init__(self, name:str, age:int, grade:int):
        self.name:str = name
        self.age:int = age
        self.grade:int = grade
    
    def save(self):
        # Connect to MySQL database
        connection = mysql.connector.connect(
            host="localhost",
            user="your_username",
            password="your_password",
            database="your_database"
        )

        # Create a cursor object
        cursor = connection.cursor()

        # Insert student data into the database
        query = "INSERT INTO students (name, age, grade) VALUES (%s, %s, %s)"
        values = (self.name, self.age, self.grade)
        cursor.execute(query, values)

        # Commit the transaction
        connection.commit()

        # Close the cursor and connection
        cursor.close()
        connection.close()

In [ ]:
# Create a student object
student = Student("John Doe", 18, "12th")

# Save the student data
student.save()

### Let's refactor the code

In [ ]:
class Student:
    def __init__(self, name:str, age:int, grade:int):
        self.name:str = name
        self.age:int = age
        self.grade:int = grade

In [ ]:
import mysql.connector

class StudentRepository:
    def __init__(self):
        self.__db_connection = None
    
    def save(self, student:Student):
        self.__create_db_connection()
        self.__insert_student_data(student)
        self.__close_db_connection
    
    def __create_db_connection(self):
        conn = mysql.connector.connect(host="localhost",
                                       user="your_username",
                                       password="your_password",
                                       database="your_database")
        self.__db_connection = conn
        
    def __close_db_connection(self):
        self.__db_connection.close()
        
    def __insert_student_data(self, student:Student):
        with self.__db_connection.cursor() as cursor:
            query = "INSERT INTO students (name, age, grade) VALUES (%s, %s, %s)"
            values = (student.name, student.age, student.grade)
            cursor.execute(query, values)
            self.__db_connection.commit()

In [ ]:
student = Student("John Doe", 18, "12th")
student_repo = StudentRepository()
student_repo.save(student)

## OCP: Open Close Principle

"Software components should be closed for modification, but open for extension"

Key points:
- Ease of adding new features 
- Lean to minimal cost of developing and testing software
- **OCP** often requires decoupling, which in turn automatically follows the **SRP**

### Example: Health Insurance 

In [ ]:
class HealthInsuranceCustomerProfile:
    def is_loyal(self) -> bool:
        # some complex business logic to determine 
        # a loyal health insurance customer
        return True # or False

In [ ]:
class InsurancePremiumDiscountCalculator:
    def calculate_discount(self, customer:HealthInsuranceCustomerProfile) -> int:
        if customer.is_loyal:
            return 20
        return 0

In [ ]:
customer_profile = HealthInsuranceCustomerProfile()
calculator = InsurancePremiumDiscountCalculator()
discount = calculator.calculate_discount(customer_profile)
print(f"The discount: {discount}")

### Problem: What if Health + Vehicle Insurance 

In [ ]:
class HealthInsuranceCustomerProfile:
    def is_loyal(self) -> bool:
        # some complex business logic to determine 
        # a loyal health insurance customer
        return True # or False

In [ ]:
class VehicleInsuranceCustomerProfile:
    def is_loyal(self) -> bool:
        # some complex business logic to determine 
        # a loyal vehicle insurance customer
        return True # or False

In [ ]:
class InsurancePremiumDiscountCalculator:
    def calculate_health_insurance_discount(self, customer:HealthInsuranceCustomerProfile) -> int:
        if customer.is_loyal:
            return 20
        return 0
    
    def calculate_vehicle_insurance_discount(self, customer:VehicleInsuranceCustomerProfile) -> int:
        if customer.is_loyal:
            return 20
        return 0

In [ ]:
calculator = InsurancePremiumDiscountCalculator()

health_customer = HealthInsuranceCustomerProfile()
health_discount = calculator.calculate_health_insurance_discount(health_customer)
print(f"health discount: {health_discount}")

vehicle_customer = VehicleInsuranceCustomerProfile()
vehicle_discount = calculator.calculate_vehicle_insurance_discount(vehicle_customer)
print(f"vehicle discount: {vehicle_discount}")

### Let's refactor the code

In [ ]:
from abc import ABC, abstractmethod

class CustomerProfile(ABC):
    @abstractmethod
    def is_loyal(self) -> bool:
        pass

In [ ]:
class HealthInsuranceCustomerProfile(CustomerProfile):    
    def is_loyal(self) -> bool:
        # some complex business logic to determine 
        # a loyal health insurance customer
        return True # or False
    
class VehicleInsuranceCustomerProfile(CustomerProfile):
    def is_loyal(self) -> bool:
        # some complex business logic to determine 
        # a loyal vehicle insurance customer
        return True # or False  

In [ ]:
class InsurancePremiumDiscountCalculator:
    def calculate_discount(self, customer:CustomerProfile) -> int:
        if customer.is_loyal:
            return 20
        return 0

In [ ]:
calculator = InsurancePremiumDiscountCalculator()

health_customer = HealthInsuranceCustomerProfile()
health_discount = calculator.calculate_discount(health_customer)
print(f"health discount: {health_discount}")

vehicle_customer = VehicleInsuranceCustomerProfile()
vehicle_discount = calculator.calculate_discount(vehicle_customer)
print(f"vehicle discount: {vehicle_discount}")

### Adding Home Insurance? No Problem

No need to modify the existing code base

In [ ]:
from abc import ABC, abstractmethod

class CustomerProfile(ABC):
    @abstractmethod
    def is_loyal(self) -> bool:
        pass
    
class HealthInsuranceCustomerProfile(CustomerProfile):    
    def is_loyal(self) -> bool:
        # some complex business logic to determine 
        # a loyal health insurance customer
        return True # or False
    
class VehicleInsuranceCustomerProfile(CustomerProfile):
    def is_loyal(self) -> bool:
        # some complex business logic to determine 
        # a loyal vehicle insurance customer
        return True # or False  
    
class InsurancePremiumDiscountCalculator:
    def calculate_discount(self, customer:CustomerProfile) -> int:
        if customer.is_loyal:
            return 20
        return 0

In [ ]:
calculator = InsurancePremiumDiscountCalculator()

health_customer = HealthInsuranceCustomerProfile()
health_discount = calculator.calculate_discount(health_customer)
print(f"health discount: {health_discount}")

vehicle_customer = VehicleInsuranceCustomerProfile()
vehicle_discount = calculator.calculate_discount(vehicle_customer)
print(f"vehicle discount: {vehicle_discount}")

Simply adding a new implementation for CustomerProfile

In [ ]:
class HomeInsuranceCustomerProfile(CustomerProfile):
    def is_loyal(self) -> bool:
        # some complex business logic to determine 
        # a loyal home insurance customer
        return True # or False  

In [ ]:
home_customer = HomeInsuranceCustomerProfile()
home_discount = calculator.calculate_discount(home_customer)
print(f"home discount: {home_discount}")

## LSP: Liskov Substitution Principle

"Objects should be replaceable with their subtypes without affecting the correctness of ther program"

Key points:
- Change the "Is-A" way of thinking
- "If it looks like a duck and quacks like a duck but it needs batteries, you probably have the wrong abstraction"

### Example: Don't foce a pinguin to fly

In [ ]:
from abc import ABC, abstractmethod

class Bird(ABC):
    @abstractmethod
    def fly(self):
        pass

In [ ]:
class Pigeon(Bird):
    def fly(self):
        print("Pigeon style flying")

In [ ]:
pigeon = Pigeon()
pigeon.fly()

In [ ]:
class Pinguin(Bird):
    def fly(self):
        raise TypeError("Piguin cannot fly")

In [ ]:
pinguin = Pinguin()
pinguin.fly()

### Example: Breaking the hierarchy

In [ ]:
class Car:
    def show_cabin_width(self):
        print("Showing cabin width")

In [ ]:
class RacingCar(Car):
#     def show_cabin_width(self):
#         raise TypeError("Racingcar doesn't have cabin")
    
#     def show_cabin_width(self):
#         self.show_cockpit_width()
        
    def show_cockpit_width(self):
        print("Showing cockpit width")

In [ ]:
cars = []
cars.append(Car())
cars.append(Car())
cars.append(RacingCar())

for car in cars:
    car.show_cabin_width()

### Let's refactor the code

In [ ]:
from abc import ABC, abstractmethod

class Vehicle(ABC):
    @abstractmethod
    def show_interior_width(self):
        pass

In [ ]:
class Car(Vehicle):
    def show_interior_width(self):
        self.show_cabin_width()
    
    def show_cabin_width(self):
        print("Showing cabin width")

In [ ]:
class RacingCar(Vehicle):
    def show_interior_width(self):
        self.show_cockpit_width()
    
    def show_cockpit_width(self):
        print("Showing cockpit width")

In [ ]:
vehicles = []
vehicles.append(Car())
vehicles.append(Car())
vehicles.append(RacingCar())

for vehicle in vehicles:
    vehicle.show_interior_width()